# Requester

This presentations goal is it to introduce the features of the `Requester` and how to configure it.

### The challenge

* I want to trigger external systems with or without field values from the currently processed event in the requests payload
* I want to enrich events by external systems

from this:

In [34]:
document = {
    'message': {
        "hostname": "H34222S3"
        }
    }

to this:

In [35]:
expected = {
    'message': {
        'hostname': 'H34222S3',
        'location': {
            'building': 'H5', 
            'floor': '3', 
            'room': '123'
        }
    }
}


### Create rule and processor

create the rule:

In [36]:
import sys
sys.path.append("../../../../../")
import tempfile
from pathlib import Path

rule_yaml = """---
filter: "message.hostname"
requester:
  target_field_mapping:
    location: message.location
  method: GET
  url: http://localhost:32000/requester_api_example.json
  
"""

rule_path = Path(tempfile.gettempdir()) / "requester"
rule_path.mkdir(exist_ok=True)
rule_file = rule_path / "requester.yml"
rule_file.write_text(rule_yaml)

153

create the processor config:

In [37]:
processor_config = {
    "cmdbrequests":{   
        "type": "requester",
        "rules": [str(rule_path)],
        }
    }

create the processor with the factory:

In [38]:
from unittest import mock
from logprep.factory import Factory

mock_logger = mock.MagicMock()
requester = Factory.create(processor_config)
requester

requester

### Process event

In [39]:
from copy import deepcopy
from tests.acceptance.util import TestServer
mydocument = deepcopy(document)

with TestServer.run_in_thread():
    print(f"before: {mydocument}")
    requester.process(mydocument)
    print(f"after: {mydocument}")
    print(mydocument == expected)


2022-12-13 02:24:31,539 urllib3.connectionpool DEBUG   : Starting new HTTP connection (1): localhost:32000
127.0.0.1 - - [13/Dec/2022 02:24:31] "GET /api.json HTTP/1.1" 200 -
2022-12-13 02:24:31,542 urllib3.connectionpool DEBUG   : http://localhost:32000 "GET /api.json HTTP/1.1" 200 97


before: {'message': {'hostname': 'H34222S3'}}
after: {'message': {'hostname': 'H34222S3', 'location': {'building': 'H5', 'floor': '3', 'room': '123'}}}
True
